In [3]:
# import required libraries
import yfinance as yf
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

# Set display options for pandas
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [32]:
# List of stock symbols for data collection
stocks = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NVDA', 'JPM', 'V', 'NFLX']

# Define the date range
start_date = '2010-01-01'
end_date = '2024-01-01'

In [33]:
# Fetch data for each stock
dataframes = []
for stock in stocks:
    try:
        df = yf.download(stock, start=start_date, end=end_date)
        df['Stock'] = stock
        dataframes.append(df)
        print(f"Downloaded data for {stock}")
    except Exception as e:
        print(f"Failed to download data for {stock}: {e}")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Downloaded data for AAPL
Downloaded data for MSFT
Downloaded data for GOOGL
Downloaded data for AMZN
Downloaded data for TSLA
Downloaded data for META
Downloaded data for NVDA
Downloaded data for JPM


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Downloaded data for V
Downloaded data for NFLX


In [34]:
# Concatenate all dataframes
data = pd.concat(dataframes)

In [35]:
# Reset index
data.reset_index(inplace=True)

In [36]:
display(data.head())

,Date,Open,High,Low,Close,Adj Close,Volume,Stock
0,2010-01-04,7.622500,7.660714,7.585000,7.643214,6.470742,493729600,AAPL
1,2010-01-05,7.664286,7.699643,7.616071,7.656429,6.481928,601904800,AAPL
2,2010-01-06,7.656429,7.686786,7.526786,7.534643,6.378825,552160000,AAPL
3,2010-01-07,7.562500,7.571429,7.466071,7.520714,6.367032,477131200,AAPL
4,2010-01-08,7.510714,7.571429,7.466429,7.570714,6.409363,447610800,AAPL


In [37]:
display(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34499 entries, 0 to 34498
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       34499 non-null  datetime64[ns]
 1   Open       34499 non-null  float64       
 2   High       34499 non-null  float64       
 3   Low        34499 non-null  float64       
 4   Close      34499 non-null  float64       
 5   Adj Close  34499 non-null  float64       
 6   Volume     34499 non-null  int64         
 7   Stock      34499 non-null  object        
dtypes: datetime64[ns](1), float64(5), int64(1), object(1)
memory usage: 2.1+ MB


None

In [25]:
# Save the data to the notebooks/data folder
data.to_csv('data/stock_data.csv', index=False)

In [42]:
# Database connection details
db_host = 'localhost'
db_port = 3306
db_name = 'stock_db'
db_user = 'stock_user'
db_password = 'calculus'

In [43]:
# make the database connection
try:
  conn=mysql.connector.connect(host=db_host,
                               user=db_user,
                               password=db_password,
                               database=db_name,
                               auth_plugin='mysql_native_password')
  print("Connection successful")
except Exception as e:
  print(e)

Connection successful


In [45]:
# If connection is successful, proceed to insert data using SQLAlchemy
if conn:
    try:
        # Create SQLAlchemy engine
        engine = create_engine(f"mysql+mysqlconnector://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}", connect_args={"auth_plugin": "mysql_native_password"}, echo=True)

        # Insert DataFrame into MySQL table
        data.to_sql('stock', con=engine, if_exists='append', index=False)
        print("Data inserted successfully")

    except Exception as e:
        print(f"Error: {e}")

    finally:
        # Close the connection
        conn.close()
        engine.dispose()

2024-06-18 01:20:14,570 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2024-06-18 01:20:14,572 INFO sqlalchemy.engine.Engine [raw sql] {}


2024-06-18 01:20:14,574 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2024-06-18 01:20:14,575 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-06-18 01:20:14,577 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2024-06-18 01:20:14,578 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-06-18 01:20:14,580 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-18 01:20:14,584 INFO sqlalchemy.engine.Engine DESCRIBE `stock_db`.`stock`
2024-06-18 01:20:14,585 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-06-18 01:20:14,749 INFO sqlalchemy.engine.Engine INSERT INTO stock (`Date`, `Open`, `High`, `Low`, `Close`, `Adj Close`, `Volume`, `Stock`) VALUES (%(Date)s, %(Open)s, %(High)s, %(Low)s, %(Close)s, %(Adj_Close)s, %(Volume)s, %(Stock)s)
2024-06-18 01:20:14,750 INFO sqlalchemy.engine.Engine [generated in 0.12745s] [{'Date': datetime.datetime(2010, 1, 4, 0, 0), 'Open': 7.622499942779541, 'High': 7.660714149475098, 'Low': 7.585000038146973, 'Close': 7.643214225769043, 'Adj_Close': 6

### Summary:

* The historical monthly stock market data (from 2010-01-01 to 2024-01-01) of 'Apple Inc.', 'Microsoft Corporation', 'Alphabet Inc.', 'Amazon.com, Inc.', 'Tesla, Inc.', 'Meta Platforms, Inc.', 'NVIDIA Corporation', 'JPMorgan Chase & Co.', 'Visa Inc.', 'Netflix, Inc.' companies are collected from Yahoo Finance website & saved into data/stock_data.csv file.
* Connection with MySQL database has been established & the same data has been inserted into a table named `stock`.